In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
import datetime as dt
import math
import random
import time

In [2]:
data=pd.read_csv('final_small.csv')  

In [3]:
data = data.drop(columns=['Unnamed: 0'])

In [4]:
data['Volume']=data['Volume'].apply(np.floor) # round down

In [5]:
data['Date'] = pd.to_datetime(data['Date'],format='%Y-%m-%d')

In [6]:
data

,Date,Open,High,Low,Close,Volume,stock,OP_HI,LO_CL,profit
0,1970-01-02,3.1254,3.1254,3.0920,3.1254,2944.0,JPM,-0.062508,-0.028774,-0.028774
1,1970-01-02,0.7383,0.7538,0.7383,0.7538,8378.0,CVX,0.000579,0.000579,0.000579
2,1970-01-02,1.3715,1.3796,1.3715,1.3796,103128.0,PG,-0.019411,-0.019411,-0.019411
3,1970-01-02,0.6104,0.6104,0.5941,0.5941,146856.0,JNJ,-0.012208,-0.011882,-0.011882
4,1970-01-02,1.4972,1.4972,1.4893,1.4972,145961.0,XOM,-0.029944,-0.021965,-0.021965
...,...,...,...,...,...,...,...,...,...,...
452541,2017-11-10,0.4937,0.4999,0.4210,0.4500,149070.0,CIE,-0.003736,0.020290,0.020290
452542,2017-11-10,1412.0000,1433.0000,1412.0000,1414.1300,44.0,HBANP,-7.450000,-26.131300,-7.450000
452543,2017-11-10,83.7900,84.0950,83.2300,83.8700,1939630.0,MSFT,-1.373850,-1.031000,-1.031000
452544,2017-11-10,3.1000,3.1800,2.9900,3.0100,2537.0,CLBS,0.017200,-0.040000,0.017200


In [61]:
Start =pd.to_datetime("1970-01-02",format='%Y-%m-%d')
End = pd.to_datetime("2017-11-10",format='%Y-%m-%d')

In [62]:
sliding_window=35
window=dt.timedelta(days=sliding_window)
balance = 1.0

In [63]:
def find_best_in_company(data):
    max_profit = float("-inf")
    buy_date,sell_date = None, None
    data=data[data.Low*1.01<balance]
    max_volume=0
    for i, row in data.iterrows():
        low = data.loc[i]["Low"]*1.01
        for j,row1 in data[data['Date'] > data.loc[i]['Date']].iterrows():
            profit = data.loc[j]["High"]*0.99 -low
            volume=min(math.ceil(balance/low)-1,min(data.loc[i]['Volume'],data.loc[j]['Volume']))
            if(volume>0 and volume*profit>max_profit):
                max_profit = volume*profit
                max_volume = volume
                buy_date=data.loc[i]['Date']
                sell_date=data.loc[j]['Date']
    return (max_profit,max_volume,buy_date,sell_date)

In [64]:
def find_best_in_window(data):
    companies = []
    for company in data.stock.unique():
        max_profit,volume,buy_date,sell_date = find_best_in_company(data[data.stock == company])
        companies.append((company,max_profit,volume,buy_date,sell_date))
    companies.sort(key=lambda tup: tup[1],reverse=True)
    return companies[0] 

In [65]:
starting_window = Start
ending_window = Start + window
moves=0
with open('C:/Users/User/Desktop/time-travel/small.txt', 'w+') as f:
    while starting_window <= End:
        print("Moves:",moves)
        print("Balance",balance,"\n")
        mask = ((data['Date'] > starting_window) & (data['Date'] <= ending_window))
        result = find_best_in_window(data.loc[mask])
        balance=balance+result[1]
        moves=moves+2
        f.write('{} {} {} {}\n'.format(result[3].date(), "buy-low", result[0], int(result[2])))
        f.write('{} {} {} {}\n'.format(result[4].date(), "sell-high", result[0], int(result[2]))) 
        starting_window=ending_window
        ending_window = starting_window + window
print("Total Moves=", moves)

Moves: 0
Balance 1.0 

Moves: 2
Balance 1.054427 

Moves: 4
Balance 1.1274980000000001 

Moves: 6
Balance 1.172266 

Moves: 8
Balance 1.2351653999999999 

Moves: 10
Balance 1.6236282 

Moves: 12
Balance 1.8224852 

Moves: 14
Balance 2.142474 

Moves: 16
Balance 2.328634 

Moves: 18
Balance 2.610098 

Moves: 20
Balance 2.844248 

Moves: 22
Balance 3.3117980000000005 

Moves: 24
Balance 3.692343 

Moves: 26
Balance 4.144737999999999 

Moves: 28
Balance 4.832982999999999 

Moves: 30
Balance 5.2484259999999985 

Moves: 32
Balance 5.607882999999999 

Moves: 34
Balance 6.326478999999998 

Moves: 36
Balance 6.751427999999998 

Moves: 38
Balance 7.3258959999999975 

Moves: 40
Balance 7.960135999999997 

Moves: 42
Balance 8.992435999999998 

Moves: 44
Balance 17.735598499999995 

Moves: 46
Balance 19.992705499999992 

Moves: 48
Balance 24.474888099999994 

Moves: 50
Balance 29.967683299999997 

Moves: 52
Balance 44.24012629999999 

Moves: 54
Balance 58.151689899999994 

Moves: 56
Balance 76.436

In [66]:
balance

21677640348.653584